# Compare the forecasts with real records

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv('.env')

# Get the values of host, user, pswd, db, and schema from the environment variables
host = os.getenv('host')
user = os.getenv('user')
pswd = os.getenv('pswd')
db = os.getenv('db')
schema = os.getenv('schema')


# Use the values as needed
engine = create_engine(
    f"postgresql://{user}:{pswd}@{host}/{db}?options=-csearch_path%3D{schema}", echo=False)
conn = engine.connect()

In [ ]:
import datetime as dt
import pytz

now = dt.datetime.now(pytz.timezone('Asia/Singapore'))
date = now.strftime("%Y-%m-%d")
time = now.strftime("%H:%M")

period = int(now.strftime("%H")) * 2 + int(now.strftime("%M")) // 30 + 1

# date = '2024-03-25' # A hard-coded value for testing
# period = 33 # A hard-coded value for testing

# print(f"Now is {date} {time} Period {period}")

In [ ]:
predicted = pd.read_sql(f"""
SELECT "Date", "Period", "Predicted_Demand"
FROM public."Predicted_Demand"
""", conn)
predicted.sort_values(by=['Date', 'Period'], inplace=True)
predicted.tail(5)

In [ ]:
earliest_date = predicted.iloc[0, 0].strftime("%Y-%m-%d")
rt_dpr = pd.read_sql(f"""
SELECT "Date", "Period", "Demand", "TCL", "Transmission_Loss"
FROM public."Real_Time_DPR"
WHERE ("Date" >= '{earliest_date}')
ORDER BY "Date", "Period"  
""", conn)
rt_dpr.sort_values(by=['Date', 'Period'], inplace=True)
rt_dpr.reset_index(drop=True, inplace=True)
rt_dpr.fillna(0, inplace=True)
# rt_dpr.head(5)

In [ ]:
def return_real(row):
    date = row['Date']
    period = row['Period']

    real = rt_dpr[(rt_dpr['Date'] == date) & (rt_dpr['Period'] == period)]
    real["total"] = real['Demand'] + real['TCL'] + real['Transmission_Loss']
    real = real["total"].values[0]
    # print(real)

    return real

predicted["Real"] = predicted.apply(lambda row: return_real(row), axis=1)
predicted["Error (R-P)"] = predicted["Real"] - predicted["Predicted_Demand"]

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

score = np.sqrt(mean_squared_error(predicted["Real"], predicted["Predicted_Demand"]))

In [ ]:
print(f"Now is {date} {time} Period {period}")
print(f"RMSE: {score}")
predicted

In [ ]:
conn.close()